In [2]:
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
import seaborn as sns
from keras import backend as K
from keras.layers.core import Dense
from keras.optimizers import Adam
from  keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
from matplotlib import pyplot as plt
from tensorflow.keras.models import load_model
%matplotlib inline

C:\Users\Vivek_Apps\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
datagen = ImageDataGenerator(
        preprocessing_function = keras.applications.mobilenet.preprocess_input,
        featurewise_center = False,
        samplewise_center=False,
        featurewise_std_normalization = False,
        samplewise_std_normalization = False,
        zca_whitening = False,
        rotation_range=45,
        width_shift_range = 0.2,
        height_shift_range = 0.2,
        horizontal_flip = False,
        vertical_flip = False,
        shear_range = 0.2)

In [5]:
train_path = "input/Dataset/train"
valid_path = "input/Dataset/valid"

In [6]:
train_batches = datagen.flow_from_directory(train_path, target_size=(224,224), classes = ['A','B','C','D','E','F','G','H','I','K','L','O'], batch_size = 43)
valid_batches = ImageDataGenerator(preprocessing_function = keras.applications.mobilenet.preprocess_input).flow_from_directory(valid_path, target_size=(224,224), classes = ['A','B','C','D','E','F','G','H','I','K','L','O'], batch_size = 3)

Found 774 images belonging to 12 classes.
Found 72 images belonging to 12 classes.


In [20]:
def plots(ims, figsize=(12,6), rows = 1, interp = False, titles = None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize = 16)
        plt.imshow(ims[i], interpolation = None if interp else'none')

In [21]:
imgs, labels = next(train_batches)

In [22]:
mobile = keras.applications.mobilenet.MobileNet()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
mobile.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
__________

In [8]:
x = mobile.layers[-6].output
predictions = Dense(12, activation='softmax')(x)
model = Model(inputs = mobile.input, outputs = predictions)

In [9]:
for layer in model.layers[: -23]:
    layer.trainable = False

In [10]:
model.compile(Adam(lr = 0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [11]:
model.fit_generator(train_batches, steps_per_epoch = 18, validation_data = valid_batches, validation_steps = 2, epochs = 10, verbose = 2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10


ValueError: Error when checking input: expected input_1 to have shape (224, 224, 3) but got array with shape (100, 100, 3)

In [ ]:
 model.save("model.h5")